In [2]:
!pip install selenium

In [3]:
# 스레드 + 크롤링
from flask import Flask, url_for, request, jsonify, redirect, Response
from flask_cors import CORS, cross_origin

import json
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import threading
import time
import logging

import pickle

In [4]:
CRAWLED_RES_SAVED_FILE = 'crawl_data/result'

In [11]:
# 기록용 스레드
def record_thread(cond):
    print("기록용 스레드 구동 시작!")
    cnt = 0
    
    while True:
        if crawl_fin:
            f = open(CRAWLED_RES_SAVED_FILE, 'wb')
            #pickle.dump(hrefURL, f)
            f.close()
            
        time.sleep(1)

In [6]:
# 관리 스레드
def thread_manager(cond):
    print("스레드 매니저 구동 시작")
    
    while True:
        with cond:
            if crawl_fin:
                cond.notifyAll()
                
            time.sleep(1)

In [7]:
# 크롤링 스레드
def crawl_thread(cond, text):
    global crawl_fin
    crawl_fin = False
    
    print("크롤링 스레드 구동 시작!")
    
    url = "https://www.youtube.com/results?search_query="
    
    driver = webdriver.Chrome(r"C:\chromedriver.exe")
    
    search_url = url + text
    
    while True:
        with cond:
            cnt = 0
            driver.get(url = search_url)
            
            body = driver.find_element_by_tag_name('body')
            
            crawl_fin = True
            cond.wait()
            crawl_fin = False

In [12]:
if __name__ == '__main__':
    cond = threading.Condition()
    crawler = threading.Thread(name = 'Crawler', target = crawl_thread, args = (cond, '시고르자브종'))
    recorder = threading.Thread(name = 'Recorder', target = record_thread, args = (cond, ))
    manager = threading.Thread(name = 'Manager', target = thread_manager, args = (cond, ))
    
    crawler.start()
    time.sleep(2)
    recorder.start()
    manager.start()

크롤링 스레드 구동 시작!
기록용 스레드 구동 시작!
스레드 매니저 구동 시작


Exception in thread Crawler:
Traceback (most recent call last):
  File "C:\Users\user2\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\user2\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-471b8dfe0bdd>", line 17, in crawl_thread
  File "C:\Users\user2\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "C:\Users\user2\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\user2\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.WebDriverException: Message: chrome not reachable
  (Session info: chrome=101.0.4951.67)

Exception in thread Crawler:
Traceback (most rece